In [3]:
import os


In [4]:
os.chdir('../')
%pwd

'c:\\Users\\User\\Python\\pool_management_system'

In [23]:
from src.constants import CONFIG_FILE_PATH
from src.utils.utils import  read_yaml, write_yaml
from src.utils.preprocess_utils import  create_yolo_data_yaml

config = read_yaml(CONFIG_FILE_PATH)


CLASSES_TXT_PATH  = config["CLASSES_TXT_PATH"]
DATA_YAML_PATH    = config["MODEL"]["DATA_YAML_PATH"]
DATA_PATH = './data'
create_yolo_data_yaml(DATA_PATH, CLASSES_TXT_PATH, DATA_YAML_PATH)



File not found: ./artifact/original_data/classes.txt


## Check Bias

In [13]:
label_dir = "data/labels"

empty_count = 0
occupied_count = 0

for label_file in os.listdir(label_dir):
    with open(os.path.join(label_dir, label_file), "r") as f:
        for line in f.readlines():
            class_id = line.split()[0]  # First number is class ID
            if class_id == "0":
                empty_count += 1
            elif class_id == "1":
                occupied_count += 1

print(f"Empty Chairs: {empty_count}")
print(f"Occupied Chairs: {occupied_count}")


Empty Chairs: 105
Occupied Chairs: 71


# Test Image

In [16]:
# Load your trained YOLOv8 model
model = YOLO("model/my_model.pt")  # Update path if needed

# Load a test image
image_path = "test_images/side_3.jpg"  
image = cv2.imread(image_path)

# Run inference
results = model(image, imgsz=image.shape[:2])

# Draw detections on the image (manually, no cropping)
for r in results:
    for box in r.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box coordinates
        class_id = int(box.cls[0])  # Class index
        confidence = box.conf[0]  # Confidence score
        
        # Class names (ensure these match your training labels)
        class_names = ["empty_chair", "occupied_chair"]
        label = f"{class_names[class_id]} {confidence:.2f}"
        
        # Draw bounding box and label
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, label, (x1, y1 - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Save and display result
output_path = "output/detected_pool_test.jpg"
cv2.imwrite(output_path, image)

cv2.imshow("YOLOv8 Detection", image)
cv2.waitKey(0)
cv2.destroyAllWindows()



WARNING  imgsz=[2750, 4133] must be multiple of max stride 32, updating to [2752, 4160]
0: 2752x4160 1 occupied-chair, 10 unoccupied-chairs, 51172.3ms
Speed: 225.4ms preprocess, 51172.3ms inference, 33.2ms postprocess per image at shape (1, 3, 2752, 4160)


## Test Video

In [ ]:
from ultralytics import YOLO
import cv2

# Load your fine-tuned model
model = YOLO("runs/train/exp/weights/best.pt")  # Update path if needed

# Load the test video
video_path = "test_videos/pool_video.mp4"
cap = cv2.VideoCapture(video_path)

# Get video properties
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define video writer to save output
out = cv2.VideoWriter("output/pool_output.mp4", 
                      cv2.VideoWriter_fourcc(*'mp4v'), fps, 
                      (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLOv8 inference on the frame
    results = model(frame)

    # Draw detections on the frame
    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box coordinates
            class_id = int(box.cls[0])  # Class index
            confidence = box.conf[0]  # Confidence score
            
            # Class names (ensure this matches your trained labels)
            class_names = ["empty_chair", "occupied_chair"]
            label = f"{class_names[class_id]} {confidence:.2f}"
            
            # Draw bounding box and label
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Write frame to output video
    out.write(frame)

    # Show frame (optional, remove for headless execution)
    cv2.imshow("YOLOv8 Pool Chair Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Cleanup
cap.release()
out.release()
cv2.destroyAllWindows()
